In [375]:
from typing import Union, Optional, Callable, Any
from typing import Tuple, List, Dict
from typing import NamedTuple

In [376]:
from collections import defaultdict

In [377]:
import os
import sys
import time
import datetime
import random

In [378]:
import numpy as np
import pandas as pd
from PIL import Image

In [379]:
import getfem as gf

In [380]:
import torch

In [381]:
# Interfaces
class Environment():
    def step(self, action):
        ...
        
    def get_state(self):
        ...
    
class QFunc():
    def __call__(self, state):
        ...
        
class State(dict):
    ...

In [382]:
class Hole():
    def __init__(self, center: Tuple[float, float], size: float) -> None:
        self.center = center
        self.size = size

In [389]:
class GridHoleBoardEnv(Environment):
    def __init__(self, size: Tuple[float, float], grid_size: Tuple[int, int]) -> None:
        self.size: Tuple[float, float] = size
        self.grid_size: Tuple[int, int] = grid_size
        self.cell_size: Tuple[float, float] = (size[0] / grid_size[0], size[1] / grid_size[1])
        # (x, y) -> size
        self.holes: Dict[Tuple[int, int], float] = \
            {(x, y): 0. for x in range(self.grid_size[0]) for y in range(self.grid_size[1])}   
        # (x, y) -> (x_coord, y_coord)
        self.holes_center: Dict[Tuple[int, int], Tuple[float, float]] = \
            {(x, y): ((x + 0.5) * self.cell_size[0], (y + 0.5) * self.cell_size[1])     
             for x in range(self.grid_size[0]) for y in range(self.grid_size[1])}
    
    def step(self, action: Tuple[Tuple[int, int], float]) -> None:
        hole_index, size_change = action
        self.holes[hole_index] += size_change
        # TODO: Size validation
        
        

In [384]:
class GridHoleBoardFemSimulator():
    def __init__(self, element_diameter: float = 2) -> None:
        self.element_diameter = element_diameter
    
    def generate_fem_mesh(self, 
                          hole_board_env: GridHoleBoardEnv, 
                          state: Optional[Dict[Tuple[int, int], float]] = None, 
                          export_mesh: bool = False) -> gf.MesherObject:
        board = gf.MesherObject('rectangle', [0., 0.], list(hole_board_env.size))
        holes: List[gf.MesherObject] = list()
        for hole_index in hole_board_env.holes:
            center = hole_board_env.holes_center[hole_index]
            if state:
                size = state[hole_index]
            else:
                size = hole_board_env.holes[hole_index]
            holes.append(gf.MesherObject('ball', list(center), size))
            
        holes_union = gf.MesherObject('union', *holes)
        mesher = gf.MesherObject('set minus', board, holes_union)
        
        print('Beginning mesh generation')
        gf.util('trace level', 2)   # No trace for mesh generation
        mesh = gf.Mesh('generate', mesher, self.element_diameter, 2)
        
        # TODO: Add Boundary to Mesh
        boundary: Dict[str, int] = dict()
            
        # Boundary of the holes
        boundary['HOLE_BOUND'] = 1
        mesh.set_region(boundary['HOLE_BOUND'], 
                        mesh.outer_faces_in_box([1., 1.], 
                                                [hole_board_env.size[0] - 1, hole_board_env.size[1] - 1]))
        
        boundary['LEFT_BOUND'] = 2
        mesh.set_region(boundary['LEFT_BOUND'], mesh.outer_faces_with_direction([-1., 0.], 0.01))        
        
        boundary['RIGHT_BOUND'] = 3
        mesh.set_region(boundary['RIGHT_BOUND'], mesh.outer_faces_with_direction([ 1., 0.], 0.01)) 
        
        boundary['TOP_BOUND'] = 4
        mesh.set_region(boundary['TOP_BOUND'], mesh.outer_faces_with_direction([0.,  1.], 0.01)) 
        
        boundary['BOTTOM_BOUND'] = 5
        mesh.set_region(boundary['BOTTOM_BOUND'], mesh.outer_faces_with_direction([0., -1.], 0.01)) 
        
        mesh.region_subtract( boundary['RIGHT_BOUND'], boundary['HOLE_BOUND'])
        mesh.region_subtract(  boundary['LEFT_BOUND'], boundary['HOLE_BOUND'])
        mesh.region_subtract(   boundary['TOP_BOUND'], boundary['HOLE_BOUND'])
        mesh.region_subtract(boundary['BOTTOM_BOUND'], boundary['HOLE_BOUND'])
                
        region_id = 7
        for hole_index in hole_board_env.holes:
            center = hole_board_env.holes_center[hole_index]
            if state:
                size = state[hole_index]
            else:
                size = hole_board_env.holes[hole_index]
            bound_key = f'HOLE{hole_index[0]}_{hole_index[1]}_BOUND'
            boundary[bound_key] = region_id
            mesh.set_region(boundary[bound_key], 
                            mesh.outer_faces_in_ball(list(center), size + 0.01 * self.element_diameter))
            if region_id == 7:
                boundary['HOLE_UNION_BOUND'] = 6
                mesh.set_region(boundary['HOLE_UNION_BOUND'], 
                            mesh.outer_faces_in_ball(list(center), size + 0.01 * self.element_diameter))
            else:
                mesh.region_merge(boundary['HOLE_UNION_BOUND'], boundary[bound_key])
            region_id += 1
            
        np.testing.assert_array_equal(mesh.region(boundary['HOLE_BOUND']), 
                                      mesh.region(boundary['HOLE_UNION_BOUND']))
            
        
        if (export_mesh):
            mesh.export_to_vtk('mesh.vtk');
            print('\nMesh generation completed.');
            
        return mesh
        
    
    def run(self) -> Any:
        pass
    
    def render_image(self) -> Any:
        pass

In [385]:
class FemImageSimilarityQFunc(QFunc):
    def __init__(self, target: Image) -> None:
        pass
    
    def __call__(self, state) -> float:
        pass

In [386]:
class Agent():
    def __init__(self, q_func: Callable) -> None:
        self.action_space: Dict[str, Callable] = dict()
        #self.experience_replay: List[...] = list()
        #self.q_table: Dict = None
        self.q_function: Callable = q_func
            
    def select_action(self) -> None:
        # Move existing Hole i with movement [+-0.5, +-0.5]
        # Add new Hole to random center
        # Remove a random Hole
        
        
        
        
        pass
            
    def step(self) -> None:
        pass

In [395]:
env = GridHoleBoardEnv(size=(80, 60), grid_size=(8, 6))


fem = GridHoleBoardFemSimulator(element_diameter=2)
fem.generate_fem_mesh(env, export_mesh=True)

Beginning mesh generation

Mesh generation completed.


message from gf_mesh_get follow:
gfMesh object in dimension 2 with 17391 points and 8236 elements



In [396]:
# Visualize
import pyvista as pv
from pyvirtualdisplay.display import Display

display = Display(visible=0, size=(1280, 1024))
display.start()
p = pv.Plotter()
m = pv.read("mesh.vtk")
#contours = m.contour()
p.add_mesh(m, show_edges=True)
#p.add_mesh(contours, color="black", line_width=1)
#p.add_mesh(m.contour(8).extract_largest(), opacity=0.1)
pts = m.points
p.camera_position = 'xy'
p.screenshot('mesh.png', transparent_background=True)
p.show(window_size=[384, 384], cpos="xy", jupyter_backend='panel')

display.stop()